In [35]:
from typing import List
import numpy as np

class Simplex:
    def __init__(self, C: List[float], A: List[float], b:List[float], accuracy:float) -> None:
        self.C_coef = np.array(C)
        self.A_coef = np.array(A)
        self.b_coef = np.array(b)
        self.accuracy = accuracy
        self.table = None
        self.optimised = False
        

    def fill_initial_table(self):
        
        self.table = np.hstack((self.A_coef, np.eye(self.A_coef.shape[0]), np.reshape(self.b_coef, (-1,1))))
        func = np.hstack((-self.C_coef, np.zeros(self.A_coef.shape[0] + 1)))
        self.table = np.vstack((self.table, func))

    def print_current_table(self):
        print(self.table)

    def is_optimised(self):
        return self.is_optimised    

    def make_iteration(self):
        if self.table is None:
            print("Table was not initialized!")
            return
        
        pivot_column = np.argmin(self.table[-1, :-1])
        if self.table[-1, :-1][pivot_column] >= -self.accuracy:
            self.optimised = True
            return
        ratios = np.divide(self.table[:-1, -1], self.table[:-1, pivot_column], out = np.full_like(self.table[:-1, -1], np.inf), where = self.table[:-1, pivot_column] > 0 )
        pivot_row = np.argmin(ratios)

        self.table[pivot_row] = self.table[pivot_row]/self.table[pivot_row][pivot_column]

        for row in range(self.table.shape[0]):
            if row != pivot_row:
                self.table[row] = self.table[row] - self.table[row][pivot_column]*self.table[pivot_row]

    def get_solution(self): 
        while not self.optimised:
            self.make_iteration()

        # Initialize solution array with the correct size: length of C_coef (number of decision variables)
        solution = np.zeros(self.C_coef.shape[0] + self.A_coef.shape[0])  # Decision vars + slack vars
        for row in range(self.A_coef.shape[0]):
        # Find the column index in this row where the value is 1
            for col in range(self.C_coef.shape[0] + self.A_coef.shape[0]):
                # Check if this column is a basic variable
                if self.table[row, col] == 1 and np.sum(self.table[:, col]) == 1:
                    # This is a basic variable column
                    solution[col] = self.table[row, -1]
                    break  # Move to the next row

        # Extract decision variables from the solution
        decision_vars = solution[:self.C_coef.shape[0]]
        max_value = self.table[-1, -1]

        return decision_vars, max_value 

The input contains:  
• A vector of coefficients of objective function - C.  
• A matrix of coefficients of constraint function - A.  
• A vector of right-hand side numbers - b.  
• The approximation accuracy.  

The output contains:  
• The string ”The method is not applicable!”  
or  
• A vector of decision variables - x.  
• Maximum (minimum) value of the objective function.

In [40]:
def get_answer(C: List[float], A: List[float], b:List[float], accuracy: float):
    simplex = Simplex(C, A, b, accuracy)
    simplex.fill_initial_table()
    answer, max_value = simplex.get_solution()
    print("Decision variables:")
    for i in range(len(answer)):
        print(f"x{i + 1} = {answer[i]}")
    print(f"Optimized objective function's value: {max_value}")

### Test #1: incorrect

In [41]:
# https://www.cuemath.com/algebra/linear-programming/
# When x1 = 4 and x2 = 8 then value of Z = 400
# x2 != 0

C = [40, 30]
A = [[1, 1], [2, 1]]
b = [12, 16]
accuracy = 0.1

get_answer(C, A, b, accuracy)

Decision variables:
x1 = 4.0
x2 = 8.0
Optimized objective function's value: 400.0


### Test #2: incorrect

In [42]:
# 33 is the maximum value of Z and it occurs at C. Thus, the solution is x = 4 and y = 5.
# we don't have x3

C = [2, 5]
A = [[1, 4], [3, 1], [1, 1]]
b = [24, 21, 9]
accuracy = 0.5

get_answer(C, A, b, accuracy)

Decision variables:
x1 = 4.0
x2 = 5.0
Optimized objective function's value: 33.0


### Test #3: incorrect

In [43]:
# We get the maximum value of Z = 27 at x1 = 0, x2 = 9 x3 = 3
# Where is x3???

C = [1, 2, 3]
A = [[1, 1, 1], [2, 1, 3]]
b = [12, 18]
accuracy = 0.7

get_answer(C, A, b, accuracy)

Decision variables:
x1 = 0.0
x2 = 8.999999999999998
x3 = 3.000000000000001
Optimized objective function's value: 27.0


### Test #4: incorrect

In [44]:
# When x1 = 0 and x2 = 8 and x3 = 20 then value of Z = 400
# x2 != 0

C = [9, 10, 16]
A = [[18, 15, 12], [6, 4, 8], [5, 3, 3]]
b = [360, 192, 180]
accuracy = 0.00001

get_answer(C, A, b, accuracy)

Decision variables:
x1 = 0.0
x2 = 8.0
x3 = 20.0
Optimized objective function's value: 400.0


### Test #5: incorrect

In [45]:
# When x1 = 0 and x2 = 225.0 and x3 = 0 and x4 = 150 then value of Z = 1050
# Where is x4???

C = [6, 2, 2.5, 4]
A = [[5, 1, 0, 2], [4, 2, 2, 1], [1, 0, 2, 1]]
b = [1000, 600, 150]
accuracy = 0.00001

get_answer(C, A, b, accuracy)

Decision variables:
x1 = 0.0
x2 = 225.0
x3 = 0.0
x4 = 150.0
Optimized objective function's value: 1050.0


### Test #6: incorrect

In [ ]:
#https://1cov-edu.ru/linejnoe-programmirovanie/simpleks-metod/primer-net-resheniya/?ysclid=m16ir708gt504218275
# Simplex Method is not applicable :c
# We go to an infinite loop

C = [4, 5, 4]
A = [[2, 3, -6], [4, 2, -4], [4, 6, -8]]
b = [240, 200, 160]
accuracy = 0.001

get_answer(C, A, b, accuracy)